## 主成分分析
- 教師無し学習
> 与えられた特徴量から新たな特徴量`主成分`を作り出し、元の特徴量よりも少ない数の変数（次元）でデータを説明する手法<br>

- 具体例
  - 変換前 ($X$):
    - 国語の点数、数学の点数、理科の点数、社会の点数、英語の点数...
    - （変数が多くて、生徒の能力が分かりにくい）
  - 変換後 ($Z$):
    - 第1主成分: 「総合的な学力」（全部の科目の相関）
    - 第2主成分: 「文系 vs 理系 の傾向」（国語が高くて数学が低い、等の差）

![Picture of PCA](images/121801.png)

1. **中心化**
- 各特徴量（列方向）の平均を計算し、引く
  - axis=0<br>
  分散で割るのは標準化

2. **共分散行列**　Covariance Matrix

$$
\Sigma = 
\begin{pmatrix}
\text{Var}(x) & \text{Cov}(x, y) & \text{Cov}(x, z) \\
\text{Cov}(y, x) & \text{Var}(y) & \text{Cov}(y, z) \\
\text{Cov}(z, x) & \text{Cov}(z, y) & \text{Var}(z)
\end{pmatrix}
$$
> 多次元であっても、共分散（二つの変数の関係）と分散を行列に並べているだけ

$$
\text{共分散行列 } \Sigma = \frac{1}{n-1} \underbrace{(X - \text{mean})^T}_{\text{転置した入力}} \cdot \underbrace{(X - \text{mean})}_{\text{入力}}
$$

* **$(X - \text{mean})^T$**: $[特徴量 \times サンプル]$ の形
* **$(X - \text{mean})$**: $[サンプル \times 特徴量]$ の形
* **掛け算の結果**: $[特徴量 \times 特徴量]$ の正方形になります。
<br>

3. **固有値分解**

In [ ]:
import tensorflow as tf

class PCATensorFlow:
    """
    TensorFlowで主成分分析（PCA）を実装してみる
    """
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None
        self.mean_ = None
        self.eigenvalues_ = None

    def fit(self, X):
        """
        モデルの学習：データの形を読み取り、重要な軸を見つける

        X=tf.cast(X, tf.float32)
            - 普通CPUは倍精度浮動小数点64bit演算で有効桁数約16桁
            - GPUは速度を求めるために32bitの単精度演算に変換させる
        
        tf.matmul(A, B, transpose_a=True)
            - 行列Aの転置行列と行列Bの行列積を計算する
            - 転置行列は内積演算を意味するので、transpose_a=True
        
        tf.linalg.eigh(cov_matrix)
            - 共分散行列の固有値と固有ベクトルを計算
                - 特性方程式を解く
        
        self.components_ = self.eigenvectors_[:, :self.n_components]
            - ,の前の:はすべての行
            - 列に関しては最初からn_components列までを選択
            - 固有ベクトルのうち、上位n_components個を選択し、新しい軸として保存
                - 分散の小さい固有ベクトルを無視することがデータの次元削除である
            > 部分空間を構成する基底ベクトルを選択する
        """
        X = tf.cast(X, tf.float32) # 単精度浮動小数点に変換
        self.mean_ = tf.reduce_mean(X, axis=0) # 平均を計算(axis=0で各特徴量、列ごとの平均)
        X_centered = X - self.mean_ # データの中心化
        
        n_samples = tf.cast(tf.shape(X)[0], tf.float32) 

        # 2.共分散行列の計算
        cov_matrix =tf.matmul(X_centered, X_centered, transpose_a=True)/ (n_samples - 1)

        # 3. 固有値分解（行列を固有値と固有ベクトルに分解）
        eigenvalues, eigenvectors = tf.linalg.eigh(cov_matrix)

        # 4. ソート（降順：大きい順に並び変えること）
        self.eigenvalues_ = eigenvalues[::-1] # 降順に並び替えた固有値
        self.eigenvectors_ = eigenvectors[:, ::-1] # 降順に並び替えた固有ベクトル

        # 5. 軸の選択
        self.components_ = self.eigenvectors_[:, :self.n_components]

    def transform(self, X):
        """
        データを主成分（新しい軸）に射影する関数
        """
        X = tf.cast(X, tf.float32)
        X_centered = X - self.mean_
        
        # 5.線形写像を実行して座標系を変換
        return tf.matmul(X_centered, self.components_)

<img src="Notes/PCA.jpg" width="500">

### データ行列 $X$

Pythonでは **「行(横)＝データサンプル」**、**「列(縦)＝特徴量**とするのが不文律

$$
X = 
\begin{pmatrix}
\text{A君の国語} & \text{A君の数学} \\
\text{B君の国語} & \text{B君の数学} \\
\text{C君の国語} & \text{C君の数学}
\end{pmatrix}
$$


音声の復元<br>
データ量の圧縮<br>

